## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [9]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [25]:
data = datasets.load_breast_cancer()

x_train, x_test, y_train, y_test = train_test_split(data.data, data.target, test_size = 0.2, random_state = 4)

clf = GradientBoostingClassifier(random_state=7)

# 先看看使用預設參數得到的結果，約為 0.9386 的 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

0.9385964912280702


In [26]:
# 設定要訓練的超參數組合
n_estimators = range(100, 500, 100)
min_samples_split = range(1, 30 , 5)
param_grid = dict(n_estimators=n_estimators, min_samples_split = min_samples_split)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="roc_auc", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   26.4s finished


In [27]:
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.990995 using {'min_samples_split': 21, 'n_estimators': 200}


In [28]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(min_samples_split=grid_result.best_params_['min_samples_split'],
                                           n_estimators=grid_result.best_params_['n_estimators'], 
                                           max_depth=1)

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

# 調整參數後約可降至 8.30 的 MSE
print(metrics.accuracy_score(y_test, y_pred))

0.956140350877193
